In [ ]:
! pip install --quiet environs cyksuid toolz psycopg2-binary typing_json backoff pyyaml facebook_business pystache

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from environs import Env

from adopt.malaria import get_confs, get_df, load_basics, window, get_db_conf
from adopt.marketing import make_audience_conf, dict_from_nested_type
from adopt.marketing import Marketing
from adopt.facebook.state import CampaignState, get_api
from adopt.malaria import load_typed_json
from adopt.marketing import AudienceConf, CampaignConf
from adopt.campaign_queries import get_campaigns_for_user, create_campaign_confs, create_campaign_for_user
from typing import List
import json

env = Env()
env.read_env('.env-vlab-vlab', override=True)
db_conf = get_db_conf(env)

In [81]:
USER = "mchatila@worldbank.org"
CAMPAIGN = "vaccination-iraq"
CAMPAIGNID = next(c['id'] for c in get_campaigns_for_user(USER, db_conf) 
                  if c['name'] == CAMPAIGN)

# create_campaign_for_user(USER, CAMPAIGN, db_conf)

In [82]:
userinfo, config, db_conf, state, m, confs = load_basics(CAMPAIGNID, env)

In [83]:
template_state = CampaignState(userinfo.token, 
                               get_api(env, userinfo.token), 
                               '282370023153169', 
                               'embed-mena-wbg-templates')



In [84]:
import pandas as pd

share_lookup = pd.read_csv('outs/wbg-distribution.csv', header=[0,1], index_col=[0])

In [85]:
from mena.strata import get_groups

groups = get_groups(template_state, share_lookup)

INFO:root:Loaded 8 adsets from campaign embed-mena-wbg-templates


# CREATING CONFS - MNM

In [4]:
USER = "dpinzonhernandez@worldbank.org"
CAMPAIGN = "vaccination-wbg"
CAMPAIGNID = next(c['id'] for c in get_campaigns_for_user(USER, db_conf) 
                  if c['name'] == CAMPAIGN)

# create_campaign_for_user(USER, CAMPAIGN, db_conf)

In [18]:
userinfo, config, db_conf, state, m, confs = load_basics(CAMPAIGNID, env)

In [88]:
images = {i['name']: i for i in state.account.get_ad_images(fields=['name', 'hash'])}

In [21]:
from adopt.facebook.update import Instruction
from adopt.malaria import run_instructions

def create_campaign(name):
    params = {
        "name": name,
        "objective": "MESSAGES",
        "status": "PAUSED",
        "special_ad_categories": [],
    }

    return Instruction("campaign", "create", params)


# run_instructions([create_campaign("vaccine_vlab_wbg")], state)

In [ ]:
# run_instructions([Instruction("campaign", "update", {"status": "PAUSED"}, "23846641567840784")], state)

In [41]:
# make creatives conf
from adopt.marketing import CreativeConf


In [9]:
AD_CAMPAIGN = 'vaccine_vlab_wbg'
AD_ACCOUNT = '246148586918843'
PAGE_ID = '103760978380374'
RESPONDENT_AUDIENCE = "vlab-embed-wbg-vaccination-respondents"
INITIAL_SHORTCODE = 'vaccwbg'
SURVEY_SHORTCODES = ['vaccar', 'vaccen']
FINISHED_QUESTION_REF = 'age_cat'
EXTRA_METADATA = {'country': 'wbg'}

In [ ]:
c = {'optimization_goal': 'LINK_CLICKS',
     'destination_type': 'MESSENGER',
     'adset_hours': 48,
     'budget': 100000.0,
     'min_budget': 100.0,
     'opt_window': 2*24,
     'end_date': '2021-02-25',
     'proportional': True,
     'page_id': PAGE_ID,
     'instagram_id': None,
     'ad_account': AD_ACCOUNT,
     'ad_campaign': AD_CAMPAIGN}

config = CampaignConf(**c)

create_campaign_confs(CAMPAIGNID, "opt", [config._asdict()], db_conf)

In [ ]:
from adopt.marketing import make_audience_conf, dict_from_nested_type
import json

audiences = [
    {
        "name": RESPONDENT_AUDIENCE,
        "shortcodes": [INITIAL_SHORTCODE],
        "subtype": "CUSTOM"
    },
]


audience_confs = [make_audience_conf(c) for c in audiences]
confs = [dict_from_nested_type(a) for a in audience_confs]

create_campaign_confs(CAMPAIGNID, "audience", confs, db_conf)

In [ ]:
from adopt.marketing import CreativeConf


def _creative_conf(name, image, body, headline, welcome_message, button_text, form):
    return {
        "name": name,
        "image": image['name'],
        "image_hash": image['hash'],
        "body": body,
        "link_text": headline,
        "welcome_message": welcome_message,
        "button_text": button_text,
        "form": form,
    }


df = pd.read_csv('outs/creative-wbg.csv')
image_confs = [{**d, 'image': images[d['image']]} for d in df.to_dict(orient='records')]

creatives = [CreativeConf(**_creative_conf(**{**c, 'form': INITIAL_SHORTCODE})) for c in image_confs]
confs = [c._asdict() for c in creatives]

create_campaign_confs(CAMPAIGNID, "creative", confs, db_conf)

In [79]:
ALL_CREATIVES = [t['name'] for t in image_confs]

def make_stratum(id_, quota, targeting):
    return { 'id': id_,
             'metadata': {'stratumid': id_, **EXTRA_METADATA},
             'facebook_targeting': targeting, 
             'creatives': ALL_CREATIVES,
             'audiences': [],
             'excluded_audiences': [RESPONDENT_AUDIENCE],
             'quota': quota,
             'shortcodes': SURVEY_SHORTCODES,
             'question_targeting': {
                 'op': 'and',
                 'vars': [
                     { 'op': 'equal',
                       'vars': [{'type': 'response', 'value': 'md:stratumid'},
                                {'type': 'constant', 'value': id_}]},
                     { 'op': 'answered',
                       'vars': [{'type': 'response', 'value': FINISHED_QUESTION_REF}]}
                 ]
             }}

In [ ]:
from adopt.marketing import make_stratum_conf

strata = [make_stratum(*g) for g in groups]
strata_data = [dict_from_nested_type(make_stratum_conf(c)) for c in strata]
create_campaign_confs(CAMPAIGNID, "stratum", strata_data, db_conf)

# TESTING

In [6]:
mal = load_basics(CAMPAIGNID, env)

In [ ]:
%%time

from adopt.malaria import update_ads_for_campaign, update_audience_for_campaign

instructions, report = update_ads_for_campaign(mal)

# instructions, report = update_audience_for_campaign(mal)

In [7]:
mal.state.total_spend

498.29

In [53]:
from adopt.malaria import hydrate_strata

userinfo, config, db_conf, state, m, confs = mal
strata = hydrate_strata(state, confs["stratum"], confs["creative"])

df = get_df(db_conf, userinfo.survey_user, strata)

In [ ]:
userinfo, config, db_conf, state, m, confs = mal
audience_confs = confs["audience"]

df = get_df(db_conf, userinfo.survey_user, audience_confs)

In [14]:
from adopt.responses import get_surveyids

surveyids = get_surveyids({'vaccwbg'}, userinfo.survey_user, db_conf)

In [ ]:
from adopt.campaign_queries import create_adopt_report

db_conf['user'] = 'adopt'
create_adopt_report(CAMPAIGNID, 'FACEBOOK_ADOPT', report, db_conf)

In [ ]:
import pandas as pd
pd.DataFrame(report).T

In [ ]:
from adopt.malaria import run_instructions

run_instructions(instructions, mal.state)